# Applied Machine Learning - Mini Challenge: Cross-Selling of Credit Cards
**Author**: Nils Fahrni

In [1]:
import sklearn
import numpy as np
import sys

sys.path.append('scripts')
from dataloader import DataLoader

## Data Preprocessing

### Data Loading

In [2]:
data_loader = DataLoader(base_path='data')
data_loader.list_datasets()

['loan', 'client', 'district', 'trans', 'account', 'card', 'order', 'disp']

In [4]:
account = data_loader.load_csv('account', parse_dates={'date': '%y%m%d'})

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,1993-01-01
1,3818,74,POPLATEK MESICNE,1993-01-01
2,704,55,POPLATEK MESICNE,1993-01-01
3,2378,16,POPLATEK MESICNE,1993-01-01
4,2632,24,POPLATEK MESICNE,1993-01-02


### Data Merging

### Data Cleaning

## Model Construction

### Identifying Existing Customers

### Identifying Existing Non-Customers

### Generating Event-Based Customer Information

## Feature Engineering

### Deriving New Features